In [ ]:
!pip install gradio
!pip install yfinance
!pip install plotly
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install re
!pip insall requests
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the re

In [ ]:
!pip install openai==0.28

# https://chatgpt.com/share/67000baa-d248-8001-9a96-27661c1ce517

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.51.0
    Uninstalling openai-1.51.0:
      Successfully uninstalled openai-1.51.0


In [ ]:
from google.colab import userdata
import pandas as pd
import openai
import gradio as gr
import yfinance as yf
import json
import os
import matplotlib.pyplot as plt
import io
from PIL import Image
import plotly.graph_objs as go  # Import Plotly for interactive charts
import re
import datetime


In [ ]:
openai.api_key = userdata.get('apopen')

In [ ]:


# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": safe_get(info, "marketCap"),
        "total_revenue": safe_get(info, "totalRevenue"),
        "gross_profit_margin": safe_get(info, "grossMargins"),
        "ebitda_margin": safe_get(info, "ebitdaMargins"),
        "operating_margin": safe_get(info, "operatingMargins"),
        "net_profit_margin": safe_get(info, "profitMargins"),
        "eps_diluted": safe_get(info, "trailingEps"),
        "pe_ratio": safe_get(info, "trailingPE"),
        "forward_pe_ratio": safe_get(info, "forwardPE")
    }
    return metrics

# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")
    current_price = history['Close'][-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns
    ytd_return = ((current_price - history['Close'][history.index >= f"{pd.Timestamp.now().year}-01-01"][0]) / history['Close'][history.index >= f"{pd.Timestamp.now().year}-01-01"][0]) * 100
    one_year_return = ((current_price - history['Close'][history.index == history.index[-252]][0]) / history['Close'][history.index == history.index[-252]][0]) * 100
    five_year_return = ((current_price - history['Close'][history.index == history.index[-1260]][0]) / history['Close'][history.index == history.index[-1260]][0]) * 100
    ten_year_return = ((current_price - history['Close'][0]) / history['Close'][0]) * 100

    performance = {
        "current_price": current_price,
        "52_week_range": f"{fifty_two_week_low} - {fifty_two_week_high}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Function to get ticker based on company name
def get_ticker_from_company_name(company_name):
    # This is a placeholder function to get a ticker from the company name
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        max_tokens=50,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        messages=[
            {"role": "system", "content": "You are a stock information assistant. Provide the stock ticker for the given company name."},
            {"role": "user", "content": company_name}
        ]
    )

    try:
        ticker = response['choices'][0]['message']['content'].strip()  # Extract the ticker symbol
        return ticker
    except:
        return None  # If no ticker is found

# Main stock chat function to handle user input and generate the report
def stock_chat(user_message):
    messages = [
        {"role": "system", "content": "You are a stock information bot. Answer only related to stock information."},
        {"role": "user", "content": user_message}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        max_tokens=50,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        messages=messages,
        functions=[
            {
                "name": "get_full_stock_report",
                "description": "Retrieve a full stock report based on the stock ticker.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {
                            "type": "string",
                            "description": "The stock ticker symbol of the company."
                        }
                    },
                    "required": ["ticker"]
                }
            }
        ]
    )

    try:
        function_call = response['choices'][0]['message'].get('function_call')

        if function_call:
            arguments = json.loads(function_call['arguments'])
            ticker = arguments.get('ticker')

            # Call the full stock report function
            report = get_full_stock_report(ticker)
            formatted_report = format_stock_report(report)
            return formatted_report

        else:
            # If no ticker symbol is provided, try to extract company name and convert to ticker
            ticker = get_ticker_from_company_name(user_message)

            if ticker:
                report = get_full_stock_report(ticker)
                formatted_report = format_stock_report(report)
                return formatted_report
            else:
                return "Sorry, I couldn't find a stock ticker for that company. Please try again with a different query."

    except Exception as e:
        return f"Error occurred: {str(e)}"


In [ ]:
stock_chat("tell me something about lenvo")

<ipython-input-7-305f2d896fac>:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = history['Close'][-1]
<ipython-input-7-305f2d896fac>:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ytd_return = ((current_price - history['Close'][history.index >= f"{pd.Timestamp.now().year}-01-01"][0]) / history['Close'][history.index >= f"{pd.Timestamp.now().year}-01-01"][0]) * 100
<ipython-input-7-305f2d896fac>:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by p

"\n**Lenovo Group Limited (LNVGY) Overview**\n- **Company Name:** Lenovo Group Limited\n- **Ticker:** LNVGY\n- **Exchange:** PNK\n- **Industry:** Computer Hardware\n- **CEO:** -\n- **Year Founded:** -\n- **Headquarters:** N/A\n\n**Description:** Lenovo Group Limited, an investment holding company, develops, manufactures, and markets technology products and services. It operates through Intelligent Devices Group, Infrastructure Solutions Group, and Solutions and Services Group segments. The company offers commercial and consumer personal computers, as well as servers and workstations; and a family of mobile Internet devices, including tablets and smartphones. In addition, it provides laptops, monitors, accessories, smart home and collaboration solutions, augmented and virtual reality, commercial internet of things, and smart infrastructure data center solutions. Further, the company manufactures and distributes IT products, computers, computer hardware, and peripheral equipment; and off